# 3 test business logic

change ${CORE_ROOT} to your path of `core`.

In [ ]:
export CORE_ROOT="${HOME}/core"

change ${PJ_ROOT} to your path of `uoa-poc2`.

In [ ]:
export PJ_ROOT="${HOME}/uoa-poc2"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/uoa-poc2
```

## load environment variables

load from `core`

In [ ]:
source ${CORE_ROOT}/docs/environments/minikube/env

load from `uoa-poc2`

In [ ]:
source ${PJ_ROOT}/docs/environments/minikube/env

## setup alias

In [ ]:
alias now="python -c 'import datetime; print(datetime.datetime.now().strftime(\"%Y-%m-%dT%H:%M:%S.%f+09:00\"))'"

## confirm the `send_cmd` command

### prepare a command to subscribe all topics

In [ ]:
echo "mosquitto_sub -h ${HOST_IPADDR} -p 1883 -d -u iotagent -P ${MQTT__iotagent} -t /#"

### subscribe all topics

_Outside of this notebook_
1. open a ternminal.
1. run the above command displayed `prepare a command to subscribe all topics`.

### publish a MQTT message to emulate gamepad action

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens | map(select(.allowed_paths[] | contains ("^/controller/.*$"))) | .[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Content-type: application/json"  http://${HOST_IPADDR}:8080/controller/api/v1/shipments/ -X POST -d @-<<__EOS__
{
}
__EOS__

example)
```
HTTP/1.1 201 Created
server: envoy
date: Tue, 06 Aug 2019 04:45:19 GMT
content-type: application/json
content-length: 21
x-envoy-upstream-service-time: 39

{"result":"success"}
```

### confirm the topic

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/I0rq9ybM2H4tiV6KYs received PUBLISH (d0, q0, r0, m0, '/delivery_robot/megarover_01/cmd', ... (158 bytes))
{"send_cmd":{"time":"2019-08-06T13:45:20.153885+09:00","cmd":"navi","waypoints":[{"x":0.503,"y":0,"z":0},{"x":3.411,"y":0,"z":0},{"x":3.411,"y":2.81,"z":0}]}}
```

### confirm delivery robot entity (megarover_01)

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/entities/${DELIVERY_ROBOT_01}/ | jq .

`send_cmd_status` is `PENDING`

example)
```json
{
  "id": "megarover_01",
  "type": "delivery_robot",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "2019-08-06T04:58:38.00Z",
    "metadata": {}
  },
  "battery": {
    "type": "object",
    "value": {
      "voltage": "11.495"
    },
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-06T04:56:07.446Z"
      }
    }
  },
  "covariance": {
    "type": "array",
    "value": [
      "0.1",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0.1",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "1.7976931348623157e+308",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "1.7976931348623157e+308",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "1.7976931348623157e+308",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0.05"
    ],
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-06T04:56:07.446Z"
      }
    }
  },
  "errors": {
    "type": "array",
    "value": "",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-06T04:56:07.446Z"
      }
    }
  },
  "mode": {
    "type": "string",
    "value": "navi",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-06T04:56:07.446Z"
      }
    }
  },
  "pose": {
    "type": "object",
    "value": {
      "point": {
        "x": "3.402",
        "y": "1.015",
        "z": "-0.002"
      },
      "angle": {
        "roll": "-0.001",
        "pitch": "0.012",
        "yaw": "1.581"
      }
    },
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-06T04:56:07.446Z"
      }
    }
  },
  "send_cmd_info": {
    "type": "commandResult",
    "value": {
      "time": "2019-08-06T13:57:59.574629+09:00",
      "received_time": "2019-07-05T07:28:26.1562279306+09:00",
      "received_cmd": "navi",
      "received_waypoints": [
        {
          "x": "0.503",
          "y": "0",
          "z": "0"
        },
        {
          "x": "3.411",
          "y": "0",
          "z": "0"
        },
        {
          "x": "3.411",
          "y": "2.81",
          "z": "0"
        }
      ],
      "result": "ack",
      "errors": ""
    },
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-06T04:57:59.700Z"
      }
    }
  },
  "send_cmd_status": {
    "type": "commandStatus",
    "value": "PENDING",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-06T04:58:38.054Z"
      }
    }
  },
  "time": {
    "type": "ISO8601",
    "value": "2019-08-06T13:56:07.309406+09:00",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-06T04:56:07.446Z"
      }
    }
  },
  "send_cmd": {
    "type": "object",
    "value": "",
    "metadata": {}
  }
}
```

### publish a MQTT message to emulate receiving command result

In [ ]:
m=$(cat << __EOS__ 
{
  "send_cmd": {
    "time": "$(now)",
    "received_time": "2019-07-05T07:28:26.1562279306+09:00",
    "received_cmd": "navi",
    "received_waypoints": [
      {
        "x": 0.503,
        "y": 0.0,
        "z": 0.0
      },
      {
        "x": 3.411,
        "y": 0.0,
        "z": 0.0
      },
      {
        "x": 3.411,
        "y": 2.81,
        "z": 0.0
      }
    ],
    "result": "ack",
    "errors": []
  }
}
__EOS__
)
mosquitto_pub -h ${HOST_IPADDR} -p 1883 -d -u iotagent -P ${MQTT__iotagent} -t /${DELIVERY_ROBOT_TYPE}/${DELIVERY_ROBOT_01}/cmdexe -m "${m}"

### confirm the topic

when executing the above command, show below messages on the opened terminal.

example)
```
Client mosq/I0rq9ybM2H4tiV6KYs received PUBLISH (d0, q0, r0, m0, '/delivery_robot/megarover_01/cmdexe', ... (449 bytes))
{
  "send_cmd": {
    "time": "2019-08-06T14:01:18.178622+09:00",
    "received_time": "2019-07-05T07:28:26.1562279306+09:00",
    "received_cmd": "navi",
    "received_waypoints": [
      {
        "x": 0.503,
        "y": 0.0,
        "z": 0.0
      },
      {
        "x": 3.411,
        "y": 0.0,
        "z": 0.0
      },
      {
        "x": 3.411,
        "y": 2.81,
        "z": 0.0
      }
    ],
    "result": "ack",
    "errors": []
  }
}
```

### confirm delivery robot entity (megarover_01)

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" http://${HOST_IPADDR}:8080/orion/v2/entities/${DELIVERY_ROBOT_01}/ | jq .

`send_cmd_status` turns to `OK` and `send_cmd_info` has the result message

example)
```json
{
  "id": "megarover_01",
  "type": "delivery_robot",
  "TimeInstant": {
    "type": "ISO8601",
    "value": "2019-08-06T05:01:18.00Z",
    "metadata": {}
  },
  "battery": {
    "type": "object",
    "value": {
      "voltage": "11.495"
    },
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-06T04:56:07.446Z"
      }
    }
  },
  "covariance": {
    "type": "array",
    "value": [
      "0.1",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0.1",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "1.7976931348623157e+308",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "1.7976931348623157e+308",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "1.7976931348623157e+308",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0",
      "0.05"
    ],
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-06T04:56:07.446Z"
      }
    }
  },
  "errors": {
    "type": "array",
    "value": "",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-06T04:56:07.446Z"
      }
    }
  },
  "mode": {
    "type": "string",
    "value": "navi",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-06T04:56:07.446Z"
      }
    }
  },
  "pose": {
    "type": "object",
    "value": {
      "point": {
        "x": "3.402",
        "y": "1.015",
        "z": "-0.002"
      },
      "angle": {
        "roll": "-0.001",
        "pitch": "0.012",
        "yaw": "1.581"
      }
    },
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-06T04:56:07.446Z"
      }
    }
  },
  "send_cmd_info": {
    "type": "commandResult",
    "value": {
      "time": "2019-08-06T14:01:18.178622+09:00",
      "received_time": "2019-07-05T07:28:26.1562279306+09:00",
      "received_cmd": "navi",
      "received_waypoints": [
        {
          "x": "0.503",
          "y": "0",
          "z": "0"
        },
        {
          "x": "3.411",
          "y": "0",
          "z": "0"
        },
        {
          "x": "3.411",
          "y": "2.81",
          "z": "0"
        }
      ],
      "result": "ack",
      "errors": ""
    },
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-06T05:01:18.311Z"
      }
    }
  },
  "send_cmd_status": {
    "type": "commandStatus",
    "value": "OK",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-06T05:01:18.311Z"
      }
    }
  },
  "time": {
    "type": "ISO8601",
    "value": "2019-08-06T13:56:07.309406+09:00",
    "metadata": {
      "TimeInstant": {
        "type": "ISO8601",
        "value": "2019-08-06T04:56:07.446Z"
      }
    }
  },
  "send_cmd": {
    "type": "object",
    "value": "",
    "metadata": {}
  }
}
```